In [1]:
import networkx as nx
import osmnx as ox
import pandas as pd 
import numpy as np
import math
from pandas import DataFrame
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
fill_df = pd.read_csv('fill_df')
fill_df

,osmid,speed_mph_mean,u,v,key,oneway,lanes,name,highway,maxspeed,length,ref,junction,bridge,access,tunnel,est_width,landuse,width,geometry
0,3996189,100.748833,23001195,359852835,0,True,3,M-40,motorway,100,96.057,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7381223 40.3618151, -3.7369952 ..."
1,3996189,100.748833,23001220,23001195,0,True,3,M-40,motorway,100,122.353,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7395663 40.3618038, -3.7381223 ..."
2,3996189,100.748833,23001944,23001220,0,True,3,M-40,motorway,100,117.894,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7409568 40.3618431, -3.7395663 ..."
3,3996189,100.748833,23001985,23001944,0,True,3,M-40,motorway,100,80.879,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7419095 40.3618888, -3.7409568 ..."
4,3996189,100.748833,23002236,23001985,0,True,3,M-40,motorway,100,88.793,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7429572 40.3619073, -3.7419095 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115985,699181046,40.089789,7549586452,260447590,0,False,1,Calle Arquitecto López Otero,tertiary,50,7.273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7307745 40.4431496, -3.7308514 ..."
115986,699286587,29.414750,201680822,6549083955,0,False,1,Avenida de la Osa Mayor,tertiary,50,17.225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7791253 40.4595704, -3.7789423 ..."
115987,699286587,29.414750,6549083955,201680822,0,False,1,Avenida de la Osa Mayor,tertiary,50,17.225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7789423 40.4596383, -3.7791253 ..."
115988,699385225,27.389875,26066630,26066631,0,False,4,Gran Vía,tertiary,30,31.299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7068988 40.4208795, -3.707185 4..."


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
fill_df.head(1000)

,osmid,speed_mph_mean,u,v,key,oneway,lanes,name,highway,maxspeed,length,ref,junction,bridge,access,tunnel,est_width,landuse,width,geometry
0,3996189,100.748833,23001195,359852835,0,True,3,M-40,motorway,100,96.057,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7381223 40.3618151, -3.7369952 ..."
1,3996189,100.748833,23001220,23001195,0,True,3,M-40,motorway,100,122.353,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7395663 40.3618038, -3.7381223 ..."
2,3996189,100.748833,23001944,23001220,0,True,3,M-40,motorway,100,117.894,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7409568 40.3618431, -3.7395663 ..."
3,3996189,100.748833,23001985,23001944,0,True,3,M-40,motorway,100,80.879,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7419095 40.3618888, -3.7409568 ..."
4,3996189,100.748833,23002236,23001985,0,True,3,M-40,motorway,100,88.793,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7429572 40.3619073, -3.7419095 ..."
5,3996189,100.748833,23002322,23002236,0,True,3,M-40,motorway,100,104.083,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.744181 40.361826, -3.7429572 40..."
6,3996190,38.552208,20952892,4920159639,0,True,3,Avenida de Arcentales,primary,50,10.158,NaN,roundabout,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.5969146 40.4305754, -3.59703 40..."
7,3996190,38.552208,21364527,20952893,0,True,3,Avenida de Arcentales,primary,50,6.582,NaN,roundabout,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.597176 40.4304978, -3.597241 40..."
8,3996190,38.552208,4920159639,21364527,0,True,3,Avenida de Arcentales,primary,50,13.667,NaN,roundabout,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.59703 40.4305503, -3.597176 40...."
9,3996191,46.386688,20952888,4570051419,0,True,2,NaN,motorway_link,70,45.631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.5965905 40.4293027, -3.5963695 ..."


In [11]:
final = pd.read_csv('final_df')

In [12]:
final.head(1000)

,osmid,speed_mph_mean,u,v,key,oneway,lanes,name,highway,maxspeed,length,ref,junction,bridge,access,tunnel,est_width,landuse,width,geometry,meanspeed,sd,z-score,congestion
0,3996189,100.748833,23001195,359852835,0,True,3,M-40,motorway,100,96.057,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7381223 40.3618151, -3.7369952 ...",94.642038,10.928605,0.558790,0
1,3996189,100.748833,23001220,23001195,0,True,3,M-40,motorway,100,122.353,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7395663 40.3618038, -3.7381223 ...",94.642038,10.928605,0.558790,0
2,3996189,100.748833,23001944,23001220,0,True,3,M-40,motorway,100,117.894,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7409568 40.3618431, -3.7395663 ...",94.642038,10.928605,0.558790,0
3,3996189,100.748833,23001985,23001944,0,True,3,M-40,motorway,100,80.879,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7419095 40.3618888, -3.7409568 ...",94.642038,10.928605,0.558790,0
4,3996189,100.748833,23002236,23001985,0,True,3,M-40,motorway,100,88.793,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.7429572 40.3619073, -3.7419095 ...",94.642038,10.928605,0.558790,0
5,3996189,100.748833,23002322,23002236,0,True,3,M-40,motorway,100,104.083,M-40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.744181 40.361826, -3.7429572 40...",94.642038,10.928605,0.558790,0
6,3996190,38.552208,20952892,4920159639,0,True,3,Avenida de Arcentales,primary,50,10.158,NaN,roundabout,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.5969146 40.4305754, -3.59703 40...",33.039047,10.435123,0.528327,0
7,3996190,38.552208,21364527,20952893,0,True,3,Avenida de Arcentales,primary,50,6.582,NaN,roundabout,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.597176 40.4304978, -3.597241 40...",33.039047,10.435123,0.528327,0
8,3996190,38.552208,4920159639,21364527,0,True,3,Avenida de Arcentales,primary,50,13.667,NaN,roundabout,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.59703 40.4305503, -3.597176 40....",33.039047,10.435123,0.528327,0
9,3996191,46.386688,20952888,4570051419,0,True,2,NaN,motorway_link,70,45.631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-3.5965905 40.4293027, -3.5963695 ...",68.018188,13.681869,-1.581034,1
